In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
import os.path
sys.path.append('/scratch/pramodkm/vqa/vqa_kazemi2017show/')
import math
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from tqdm import tqdm

import config
import data
import model_IG
import utils

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# Load pretrained model
log = torch.load('/scratch/pramodkm/vqa/vqa_kazemi2017show/logs/2017-08-04_00.55.19.pth')
tokens = len(log['vocab']['question']) + 1

net = torch.nn.DataParallel(model_IG.Net(tokens))
net.load_state_dict(log['weights'])

In [4]:
# Load vocabulary
with open(config.vocabulary_path, 'r') as fd:
    vocab_json = json.load(fd)

In [5]:
# Create embedding matrix
question_emb_lookup = log['weights']['module.text.embedding.weight']
embedding = nn.Embedding(question_emb_lookup.shape[0], question_emb_lookup.shape[1], padding_idx=0)
embedding.weight.data = question_emb_lookup

In [6]:
# Load validation data
val_loader = data.get_loader(val=True)
LOADER=val_loader
PREFIX="val"

In [13]:
tq = tqdm(LOADER, desc='{} E{:03d}'.format(PREFIX, 0), ncols=0)
net.eval()
answ = []
idxs = []
accs = []
num_batches = 0
for v, q, a, idx, q_len in tq:
        var_params = {
            'volatile': True,
            'requires_grad': False,
        }
        v = Variable(v.cuda(async=True), **var_params)
        q = Variable(q.cuda(async=True), **var_params)
        a = Variable(a.cuda(async=True), **var_params)
        q_len = Variable(q_len.cuda(async=True), **var_params)
        
        q = embedding(q)
        
        out = net(v, q, q_len)
        acc = utils.batch_accuracy(out.data, a.data).cpu()
        
        _, answer = out.data.cpu().max(dim=1)
        answ.append(answer.view(-1))
        accs.append(acc.view(-1))
        idxs.append(idx.view(-1).clone())
        print(acc.mean())
        num_batches += 1
        if num_batches == 10:
            break

answ = list(torch.cat(answ, dim=0))
accs = list(torch.cat(accs, dim=0))
idxs = list(torch.cat(idxs, dim=0))


val E000:   0% 0/950 [00:00<?, ?it/s]
/scratch/pramodkm/vqa/vqa_kazemi2017show/model_IG.py:135: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention = F.softmax(attention)
val E000:   0% 1/950 [00:01<27:06,  1.71s/it]

0.6289062523283064
0.6773437517695129


val E000:   1% 5/950 [00:02<06:36,  2.38it/s]

0.6242187512107193
0.6109375031664968
0.571093752514571


val E000:   1% 7/950 [00:02<05:05,  3.09it/s]

0.5929687530733645
0.6015625027939677


val E000:   1% 8/950 [00:02<04:42,  3.33it/s]

0.5867187534458935
0.6757812523283064
0.6140625020489097


In [ ]:
embedding = nn.Embedding(question_emb_lookup.shape[0], question_emb_lookup.shape[1], padding_idx=0)

In [ ]:
embedding.weight.data = question_emb_lookup

In [10]:
q*0.5

Variable containing:
( 0 ,.,.) = 
 -3.2798e-02  5.1392e-02  1.6251e-02  ...  -1.2319e-02 -8.2559e-02  3.3580e-02
  1.6407e-03  2.5175e-03 -1.6903e-02  ...   1.5478e-02  9.4804e-03  1.0341e-02
 -1.1436e-01 -7.8576e-02  1.3111e-01  ...   6.0400e-03 -3.7864e-02 -5.7790e-02
                 ...                   ⋱                   ...                
 -7.6968e-03 -2.3634e-03 -5.5749e-03  ...   7.1667e-04  4.6250e-03 -3.2087e-03
 -7.6968e-03 -2.3634e-03 -5.5749e-03  ...   7.1667e-04  4.6250e-03 -3.2087e-03
 -7.6968e-03 -2.3634e-03 -5.5749e-03  ...   7.1667e-04  4.6250e-03 -3.2087e-03

( 1 ,.,.) = 
 -3.7956e-02 -2.9700e-02 -1.1624e-02  ...   1.5651e-02  4.8440e-03  5.7062e-02
  1.6407e-03  2.5175e-03 -1.6903e-02  ...   1.5478e-02  9.4804e-03  1.0341e-02
  5.8533e-02 -5.8102e-02 -2.0755e-01  ...  -2.7952e-02  9.1103e-02  1.8481e-02
                 ...                   ⋱                   ...                
 -7.6968e-03 -2.3634e-03 -5.5749e-03  ...   7.1667e-04  4.6250e-03 -3.2087e-03
 -7.